In [71]:
from pyspark import SparkContext
from pyspark.sql import SparkSession,SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [72]:
rec_df     = spark.read.json('../results/usersrec/')
movies_df  = spark.read.csv('../ml-latest/movies.csv',header=True,inferSchema=True)
ratings_df = spark.read.csv('../ml-latest/ratings.csv',header=True,inferSchema=True)

In [74]:
rec_df.printSchema()
#movies_df.printSchema()
#ratings_df.printSchema()
rec_df.show()

root
 |-- itemId: string (nullable = true)
 |-- itemIndex: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: string (nullable = true)
 |-- userIndex: long (nullable = true)

+------+---------+---------+------+---------+
|itemId|itemIndex|   rating|userId|userIndex|
+------+---------+---------+------+---------+
|153034|    47502|6.8999233| 40992|      145|
|167830|    49074|6.3971386| 40992|      145|
|179845|    50646| 6.155308| 40992|      145|
|184195|    51404| 6.089066| 40992|      145|
|124061|    44864| 6.089066| 40992|      145|
|129683|    45372|5.9753647| 40992|      145|
|148408|    47109| 5.972332| 40992|      145|
|159562|    48183|5.9317083| 40992|      145|
|183363|    51259|5.8757367| 40992|      145|
|146940|    46980| 5.861775| 40992|      145|
|153034|    47502|6.5650244|251665|      200|
|167830|    49074|6.4874845|251665|      200|
|124061|    44864|5.8417153|251665|      200|
|184195|    51404|5.8417153|251665|      200|
|179845|    50646|5.

In [75]:
rec_movies_df = rec_df.join(movies_df, rec_df.itemId == movies_df.movieId, 'inner')

In [76]:
watched_movies_df = ratings_df.join(movies_df, ratings_df.movieId == movies_df.movieId, 'inner')\
                          .drop(ratings_df.movieId)

In [77]:
# 16576 - 2136 - 8433
varuser = 8433

### Recommended Movies

In [79]:
rec_movies_df.select('userId','movieId', 'title', 'genres', 'rating') \
             .filter("userId in(" + str(varuser) + ")").sort(rec_movies_df.rating.desc()).show(truncate=False)

+------+-------+--------------------------------------------+--------------------+---------+
|userId|movieId|title                                       |genres              |rating   |
+------+-------+--------------------------------------------+--------------------+---------+
|8433  |153034 |D (2005)                                    |Crime|Drama|Thriller|6.4504957|
|8433  |150308 |I Passed for White (1960)                   |Drama               |5.5751705|
|8433  |77442  |Darkness and Light (Hei an zhi guang) (1999)|Drama               |5.5742846|
|8433  |184195 |Humpty Sharma Ki Dulhania (2014)            |Comedy|Drama|Romance|5.553271 |
|8433  |124061 |Rapture (1965)                              |Drama               |5.553271 |
|8433  |147076 |Twilight for the Gods (1958)                |War                 |5.430531 |
|8433  |129767 |The Police Serve the Citizens? (1973)       |Action|Crime        |5.409882 |
|8433  |183363 |Impasse du désir (2010)                     |Thriller 

### Watched Movies

In [78]:
watched_movies_df.select('userId','movieId', 'title', 'genres') \
                 .filter("userId in(" + str(varuser) + ")").show(truncate=False)

+------+-------+---------------------------------------------------------+-----------------------------------------------+
|userId|movieId|title                                                    |genres                                         |
+------+-------+---------------------------------------------------------+-----------------------------------------------+
|8433  |47     |Seven (a.k.a. Se7en) (1995)                              |Mystery|Thriller                               |
|8433  |150    |Apollo 13 (1995)                                         |Adventure|Drama|IMAX                           |
|8433  |153    |Batman Forever (1995)                                    |Action|Adventure|Comedy|Crime                  |
|8433  |161    |Crimson Tide (1995)                                      |Drama|Thriller|War                             |
|8433  |165    |Die Hard: With a Vengeance (1995)                        |Action|Crime|Thriller                          |
|8433  |231    |